In [1]:
### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict

# DATABASE Connections

# Chemistry Database
db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print(datetime.now())

PostgreSQL database version: ('PostgreSQL 12.5 (Ubuntu 12.5-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
2021-01-17 22:36:23.982410


In [3]:
### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###

import collections
import fingerprinter as fpt

# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
# sql = 'SELECT "Structure_ID", "SMILES" FROM public."Structures"' 
sql = 'SELECT s."Structure_ID", s."SMILES" FROM public."Structures" s \
WHERE s."Structure_ID" IN (SELECT f."Structure_ID" FROM public."Structures" f \
WHERE NOT EXISTS (SELECT 1 FROM public."Structure_Fingerprint" fp WHERE f."Structure_ID" = fp."Structure_ID"))' 
cur.execute(sql) 

mollist = cur.fetchall()
print('= = = Molecules without Fingerprints = = =')
print(len(mollist))
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
        # insert fingerprints for new molecules into structure_fingerprints

print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
# insert new SMILES into Fingerprint Table
        
print()        
print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        


= = = Molecules without Fingerprints = = =
0
[]

All Fingerprints by molecule
[]

{}
0 SMILES

defaultdict(<class 'list'>, {})
0 fingerprints


In [12]:
### Step 5: Update Shannon Entropies ###


# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT sf."Structure_ID", sf."FP_ID", sf."FP_Count", fp."FP_Shannon" FROM public."Structure_Fingerprint" sf \
JOIN public."Fingerprints" fp ON sf."FP_ID" = fp."Fingerprint_ID" WHERE fp."FP_Type" = 1'
cur.execute(sql)
fp_list = cur.fetchall()
# print(fp_list)

fp_dict = collections.defaultdict(list)

for m in fp_list:
    print(m, m[1],m[2])
    fp_dict[m[1]].append(m[2])

n_fp = 0
print(len(fp_dict),'fingerprints')

for fpb, i in fp_dict.items():
    n_fp = max(n_fp, len(i))
    # print (fpb, i, n)

sql = 'UPDATE public."Fingerprints" SET "FP_Shannon" = %s WHERE "Fingerprint_ID" = %s'
cur = db_chem.cursor()
for fpb, i in fp_dict.items():
    ns = netprepper.shannon(i,n_fp)
    # print(fpb,' - ',netprepper.shannon(i,n))
    print(fpb,' - ',ns[0],' - ',ns[1])
    cur.execute(sql, (ns[1], fpb))
db_chem.commit()

# insert/update entropy table


Net_Prepper_Imported
(10, 10, 1, 0.2711893730418441) 10 1
(10, 7, 1, 0.2711893730418441) 7 1
(10, 8, 1, 1.0579054247036734) 8 1
(10, 9, 1, 0.6901856760188042) 9 1
(10, 11, 1, 1.0123308391031736) 11 1
(10, 12, 1, 0.2711893730418441) 12 1
(11, 5, 1, 0.2711893730418441) 5 1
(11, 13, 1, 0.4293230219306162) 13 1
(11, 8, 1, 1.0579054247036734) 8 1
(11, 9, 1, 0.6901856760188042) 9 1
(11, 14, 1, 0.2711893730418441) 14 1
(11, 11, 1, 1.0123308391031736) 11 1
(13, 15, 1, 0.2711893730418441) 15 1
(13, 8, 1, 1.0579054247036734) 8 1
(13, 9, 2, 0.6901856760188042) 9 2
(13, 11, 2, 1.0123308391031736) 11 2
(14, 16, 1, 0.4293230219306162) 16 1
(14, 9, 2, 0.6901856760188042) 9 2
(14, 17, 2, 0.2711893730418441) 17 2
(14, 18, 1, 0.2711893730418441) 18 1
(15, 19, 1, 0.2711893730418441) 19 1
(15, 20, 2, 0.2711893730418441) 20 2
(15, 16, 1, 0.4293230219306162) 16 1
(15, 8, 2, 1.0579054247036734) 8 2
(15, 9, 2, 0.6901856760188042) 9 2
(15, 21, 1, 0.2711893730418441) 21 1
(15, 11, 2, 1.0123308391031736) 11 2
(1

In [6]:
# Generate Fingerprint SMILES

from rdkit import Chem
from rdkit.Chem import AllChem

def FingerprintToSmiles(m, s):
    fp_sm = []
    bi = {}
    fp = AllChem.GetMorganFingerprint(m,s, bitInfo=bi)
    # print('FPSM : ', bi)
    for f in bi:
        # print('K:', f,' V:', bi[f])
        a = bi[f][0][0]
        r = bi[f][0][1]
        # print(f, a, r)
        if r > 0:
            env = Chem.FindAtomEnvironmentOfRadiusN(m,r,a)
            amap={}
            submol=Chem.PathToSubmol(m,env,atomMap=amap)
            sm =Chem.MolToSmiles(submol)
        else:
            am = m.GetAtomWithIdx(a)
            sm = am.GetSymbol()
            if am.GetIsAromatic():
                sm = sm.lower()
        fp_sm.append((f,sm))
        # print(f,' - ',sm)
        # print(f,' - ',len(v),' - ',v,' - ',a,' - ',r,' - ',sm)
    return fp_sm

In [ ]:
# Insert novel structures after structure check

smi_in = ("CCOCC", "CCNCC")

db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'INSERT INTO public."Structures"("SMILES") VALUES(%s);'
#print(sql)
for i in smi_in:   
    smi = (i,)
    # print(smi)
    try:
        cur.execute(sql, smi)
        print('Inserted:', i)
    except:
        print("Structure Exist: ", i)
        cur.close()
        db_chem.close()
        db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
        cur = db_chem.cursor()
        sql = 'INSERT INTO public."Structures"("SMILES") VALUES(%s);'




In [1]:
import chemistry_psql as chemps

smi_in = ("CCOCC", "CCNCC")

chemps.SmilesInsert(smi_in)

Structure Exist:  CCOCC
Inserted: CCNCC


In [11]:
### Step 3: Insert new fingerprints into fingerprint table ###

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public."Fingerprints"("FP_Smiles", "FP_Number", "FP_Type") VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        print(fpb,' - ',v, smi_fl, "inserted")



No new fingerprint inserts



In [8]:
### Step 4: Insert Fingerprints for New Molecules ###


# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public."Structure_Fingerprint"("Structure_ID","FP_ID","FP_Count") VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    # cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    



No new structure/fingerprint inserts

